In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
import sys
sys.path.append('/home/chenani/ownCloud/Workspaces/Eclipse/dataAnalysis/Sleep-current/src/')

In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as pl
import numpy as np
import pickle as pkl
import os
import seaborn as sns
import pandas as pd
import fnmatch
from scipy.stats import binom_test

/home/chenani/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
def locate(pattern, root=os.curdir):
    '''Locate all files matching supplied filename pattern in and below
        supplied root directory.
    '''
    for path, dirs, files in os.walk(os.path.abspath(root)):
        for filename in fnmatch.filter(files, pattern):
            yield [path,filename]
def findClosestPoint(values,array):
    values = np.array(values)
    array = np.array(array)
    idx = np.array([np.square(array - value).argmin() for value in values])
    return idx,array[idx]
def sigCounter(df,min_N = 3):
    dbMinN = df[(df.lSub>min_N)|(df.rSub>min_N)]
    nT = dbMinN.shape[0]
    nS = df[(df.Lpv<0.05)|(df.Rpv<0.05)].shape[0]
    try:
        prcnt = 1e2*nS/nT
    except ZeroDivisionError:
        prcnt = None
    #print nT,nS,binom_test(nS,nT,0.05)
    #print 1e2*nS/nT
    return nT,nS,prcnt

In [22]:
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/'

In [23]:
slpDB = np.array(sorted([os.path.join(item[0],item[1])for item in locate('SleepEpoch*.db',dataFolder)]))[:]
roDBs = sorted([os.path.join(item[0],item[1])for item in locate('*roDB*.db',dataFolder)])
prDBs = sorted([os.path.join(item[0],item[1])for item in locate('*prDB*.db',dataFolder)])
awROs = sorted([os.path.join(item[0],item[1])for item in locate('*RO.db',dataFolder)])
awPRs = sorted([os.path.join(item[0],item[1])for item in locate('*PR.db',dataFolder)])

In [24]:
slpDB,prDBs,roDBs,awROs,awPRs

(array([ '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/sleepDB/SleepEpoch-Rat434Lesion-Day1.db'], 
       dtype='|S134'),
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/sleepDB/434-prDB-D1.db'],
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/sleepDB/434-roDB-D1.db'],
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/awakeReplays-434-D1-RO.db'],
 ['/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/lesion/Rat434Lesion/dataFiles/awakeReplays-434-D1-PR.db'])

In [25]:
print '--------------------------------------------RANK-ORDER-------------------------------------------------'
for slp,db in zip(slpDB,roDBs):
    print '+++++++++++++++++++++++++++'
    print db.split('-')[-1]
    sldf = pd.read_pickle(slp)
    sldf = sldf.replace('sleep05','sleep01')
    sldf = sldf.replace('sleep06','sleep02')
    PRE_durarion = np.sum(sldf[(sldf.session=='sleep01')&(sldf.epoch!='AWAKE')].t1-sldf[(sldf.session=='sleep01')&(sldf.epoch!='AWAKE')].t0)
    POS_duration = np.sum(sldf[(sldf.session=='sleep02')&(sldf.epoch!='AWAKE')].t1-sldf[(sldf.session=='sleep02')&(sldf.epoch!='AWAKE')].t0)
    th_PRE_duration  = np.sum(sldf[(sldf.session=='sleep01')&(sldf.epoch=='REM')].t1-sldf[(sldf.session=='sleep01')&(sldf.epoch=='REM')].t0)
    del_PRE_duration = np.sum(sldf[(sldf.session=='sleep01')&(sldf.epoch=='SWS')].t1-sldf[(sldf.session=='sleep01')&(sldf.epoch=='SWS')].t0)
    th_POST_duration  = np.sum(sldf[(sldf.session=='sleep02')&(sldf.epoch=='REM')].t1-sldf[(sldf.session=='sleep02')&(sldf.epoch=='REM')].t0)
    del_POST_duration = np.sum(sldf[(sldf.session=='sleep02')&(sldf.epoch=='SWS')].t1-sldf[(sldf.session=='sleep02')&(sldf.epoch=='SWS')].t0)
    print PRE_durarion,POS_duration,del_PRE_duration,th_PRE_duration,del_POST_duration,th_POST_duration
    df   = pd.read_pickle(db)
    df = df.replace('sleep05','sleep01')
    df = df.replace('sleep06','sleep02')
    dfPRE   = df[df.session=='sleep01']
    dfPOST  = df[df.session=='sleep02']
    delPRE  = dfPRE[dfPRE.epoch=='sws']
    thePRE  = dfPRE[dfPRE.epoch=='rem']
    delPOST = dfPOST[dfPOST.epoch=='sws']
    thePOST = dfPOST[dfPOST.epoch=='rem']
    #####Countimg and rating
    preCounts = np.array(sigCounter(dfPRE,4))
    postCounts = np.array(sigCounter(dfPOST,4))
    del_preC  = np.array(sigCounter(delPRE,4))
    del_postC = np.array(sigCounter(delPOST,4))
    th_preC  = np.array(sigCounter(thePRE,4))
    th_postC = np.array(sigCounter(thePOST,4))
    print 'PRE---- Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt = %0.4f , Rs = %f ' %(preCounts[0],preCounts[1],preCounts[2],
                                                                                       (preCounts[0]/PRE_durarion),preCounts[1]/PRE_durarion)
    print 'PRE--DLT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_del = %0.4f, Rs_del = %f'%(del_preC[0],del_preC[1],del_preC[2],
                                                                                            (del_preC[0]/del_PRE_duration),(del_preC[1]/del_PRE_duration))
    print 'PRE--THT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_tht = %0.4f, Rs_tht = %f' %(th_preC[0],th_preC[1],th_preC[2],
                                                                                              (th_preC[0]/th_PRE_duration),(th_preC[1]/th_PRE_duration))
    print '############################'
    print 'POST---- Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt = %0.4f , Rs = %f ' %(postCounts[0],postCounts[1],postCounts[2],
                                                                                        (postCounts[0]/POS_duration),postCounts[1]/ POS_duration)
    print 'POST--DLT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_del = %0.4f, Rs_del = %f'%(del_postC[0],del_postC[1],del_postC[2],
                                                                                            (del_postC[0]/del_POST_duration),(del_postC[1]/del_POST_duration))
    print 'POST--THT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_tht = %0.4f, Rs_tht = %f' %(th_postC[0],th_postC[1],th_postC[2],
                                                                                              (th_postC[0]/th_POST_duration),(th_postC[1]/th_POST_duration))
print '--------------------------------------------PERSISTANCE-------------------------------------------------'
for slp,db in zip(slpDB,prDBs):
    print '+++++++++++++++++++++++++++'
    print db.split('-')[-1]
    sldf = pd.read_pickle(slp)
    sldf = sldf.replace('sleep05','sleep01')
    sldf = sldf.replace('sleep06','sleep02')
    PRE_durarion = np.sum(sldf[(sldf.session=='sleep01')&(sldf.epoch!='AWAKE')].t1-sldf[(sldf.session=='sleep01')&(sldf.epoch!='AWAKE')].t0)
    POS_duration = np.sum(sldf[(sldf.session=='sleep02')&(sldf.epoch!='AWAKE')].t1-sldf[(sldf.session=='sleep02')&(sldf.epoch!='AWAKE')].t0)
    th_PRE_duration  = np.sum(sldf[(sldf.session=='sleep01')&(sldf.epoch=='REM')].t1-sldf[(sldf.session=='sleep01')&(sldf.epoch=='REM')].t0)
    del_PRE_duration = np.sum(sldf[(sldf.session=='sleep01')&(sldf.epoch=='SWS')].t1-sldf[(sldf.session=='sleep01')&(sldf.epoch=='SWS')].t0)
    th_POST_duration  = np.sum(sldf[(sldf.session=='sleep02')&(sldf.epoch=='REM')].t1-sldf[(sldf.session=='sleep02')&(sldf.epoch=='REM')].t0)
    del_POST_duration = np.sum(sldf[(sldf.session=='sleep02')&(sldf.epoch=='SWS')].t1-sldf[(sldf.session=='sleep02')&(sldf.epoch=='SWS')].t0)
    df   = pd.read_pickle(db)
    df = df.replace('sleep05','sleep01')
    df = df.replace('sleep06','sleep02')
    dfPRE   = df[df.session=='sleep01']
    dfPOST  = df[df.session=='sleep02']
    delPRE  = dfPRE[dfPRE.epoch=='sws']
    thePRE  = dfPRE[dfPRE.epoch=='rem']
    delPOST = dfPOST[dfPOST.epoch=='sws']
    thePOST = dfPOST[dfPOST.epoch=='rem']
    #####Countimg and rating
    preCounts = np.array(sigCounter(dfPRE))
    postCounts = np.array(sigCounter(dfPOST))
    del_preC  = np.array(sigCounter(delPRE))
    del_postC = np.array(sigCounter(delPOST))
    th_preC  = np.array(sigCounter(thePRE))
    th_postC = np.array(sigCounter(thePOST))
    print 'PRE---- Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt = %0.4f , Rs = %f ' %(preCounts[0],preCounts[1],preCounts[2],
                                                                                       (preCounts[0]/PRE_durarion),preCounts[1]/PRE_durarion)
    print 'PRE--DLT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_del = %0.4f, Rs_del = %f'%(del_preC[0],del_preC[1],del_preC[2],
                                                                                            (del_preC[0]/del_PRE_duration),(del_preC[1]/del_PRE_duration))
    print 'PRE--THT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_tht = %0.4f, Rs_tht = %f' %(th_preC[0],th_preC[1],th_preC[2],
                                                                                              (th_preC[0]/th_PRE_duration),(th_preC[1]/th_PRE_duration))
    print '############################'
    print 'POST---- Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt = %0.2f , Rs = %f ' %(postCounts[0],postCounts[1],postCounts[2],
                                                                                        (postCounts[0]/POS_duration),postCounts[1]/ POS_duration)
    print 'POST--DLT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_del = %0.4f, Rs_del = %f'%(del_postC[0],del_postC[1],del_postC[2],
                                                                                            (del_postC[0]/del_POST_duration),(del_postC[1]/del_POST_duration))
    print 'POST--THT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_tht = %0.4f, Rs_tht = %f' %(th_postC[0],th_postC[1],th_postC[2],
                                                                                              (th_postC[0]/th_POST_duration),(th_postC[1]/th_POST_duration))

--------------------------------------------RANK-ORDER-------------------------------------------------
+++++++++++++++++++++++++++
D1.db
979.259 2028.829 299.586 679.673 1150.451 878.378
PRE---- Nt= 432, Ns= 29 percentage= 6.71296296296 --- Rt = 0.4411 , Rs = 0.029614 
PRE--DLT Nt= 84, Ns= 5 percentage= 5.95238095238 --- Rt_del = 0.2804, Rs_del = 0.016690
PRE--THT Nt= 255, Ns= 19 percentage= 7.45098039216 --- Rt_tht = 0.3752, Rs_tht = 0.027955
############################
POST---- Nt= 562, Ns= 73 percentage= 12.9893238434 --- Rt = 0.2770 , Rs = 0.035981 
POST--DLT Nt= 120, Ns= 20 percentage= 16.6666666667 --- Rt_del = 0.1043, Rs_del = 0.017384
POST--THT Nt= 299, Ns= 21 percentage= 7.02341137124 --- Rt_tht = 0.3404, Rs_tht = 0.023908
--------------------------------------------PERSISTANCE-------------------------------------------------
+++++++++++++++++++++++++++
D1.db
PRE---- Nt= 593, Ns= 88 percentage= 14.8397976391 --- Rt = 0.6056 , Rs = 0.089864 
PRE--DLT Nt= 114, Ns= 12 percentag

In [19]:
print '--------------------------------------------RANK-ORDER-------------------------------------------------'
for db in awROs:
    print db.split('/')[-1]
    df = pd.read_pickle(db)
    print 'RUN------- count are %s total, %s significant with percentage: %s' %sigCounter(df,4)
    
print '--------------------------------------------PERSISTANCE-------------------------------------------------'
for db in awPRs:
    print db.split('/')[-1]
    df = pd.read_pickle(db)
    print 'RUN------- count are %s total, %s significant with percentage: %s' %sigCounter(df)

--------------------------------------------RANK-ORDER-------------------------------------------------
awakeReplays-3837-D1-RO.db
RUN------- count are 104 total, 13 significant with percentage: 12.5
awakeReplays-3837-D3-RO.db
RUN------- count are 55 total, 0 significant with percentage: 0.0
awakeReplays-3837-D4-RO.db
RUN------- count are 0 total, 0 significant with percentage: None
--------------------------------------------PERSISTANCE-------------------------------------------------
awakeReplays-3837-D1-PR.db
RUN------- count are 159 total, 29 significant with percentage: 18.2389937107
awakeReplays-3837-D3-PR.db
RUN------- count are 92 total, 19 significant with percentage: 20.652173913
awakeReplays-3837-D4-PR.db
RUN------- count are 1 total, 1 significant with percentage: 100.0


In [277]:
animalPath  = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3754/LinearTrack_07-09-15_Day3/'
lfpPaths = sorted([os.path.join(item[0],item[1])for item in locate('CSC2.lfp',animalPath)])
lfpPaths

['/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3754/LinearTrack_07-09-15_Day3/01-sleep1/CSC2.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3754/LinearTrack_07-09-15_Day3/02-begin1/CSC2.lfp',
 '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3754/LinearTrack_07-09-15_Day3/03-sleep2/CSC2.lfp']

In [278]:
duration = []
for pth in lfpPaths:
    duration.append(pkl.load(open(pth,'rb')).time_parameters()[1]/1e3)
    print duration

[3602.944]
[3602.944, 1801.216]
[3602.944, 1801.216, 3614.464]


In [282]:
10/ duration[2],132/ duration[2]

(0.0027666619448969473, 0.0365199376726397)

In [445]:
animalPath  = '/home/chenani/dataWork/ali/MECLesion_SleepData/Marta/Rat3839/'
trajPaths = np.array(sorted([os.path.join(item[0],item[1])for item in locate('*.traj',animalPath)]))
dataFolder = '/home/chenani/ownCloud/IpythonNotebooks/Sleep/MEC_Lesion_Project/control/Rat3839Control/dataFiles/'
ltPaths = np.array(sorted([os.path.join(item[0],item[1])for item in locate('linearTrack*.db',dataFolder)]))

In [449]:
restTime = []
for pth in zip(ltPaths[2:],trajPaths[2:]):
    db = pkl.load(open(pth[0]))
    N = pkl.load(open(pth[1],'rb')).times.size
    dd = pkl.load(open(pth[1],'rb')).t_stop - pkl.load(open(pth[1],'rb')).t_start
    Nitp = np.arange(0,N,0.15).size
    restTime.append(1.0*(Nitp-np.diff(db,axis=1)[:,0].sum())/Nitp*dd)
    

In [24]:
sldf[(sldf.session=='sleep01')&(sldf.epoch!='AWAKE')].t0.sum()

201119.24100000001

In [32]:
print 'PRE--DLT Nt= %0.0f, Ns= %0.0f percentage= %s --- Rt_del = %0.2f, Rs_del = %f'%(del_preC[0],del_preC[1],del_preC[2],(del_preC[0]/del_PRE_duration),(del_preC[1]/del_PRE_duration))

PRE--DLT Nt= 0, Ns= 0 percentage= None --- Rt_del = 0.00, Rs_del = 0.000000


In [21]:
dfg = pd.read_pickle(awROs[2])
dfg[(dfg.lSub>3)|(dfg.rSub>3)]

,ti,tf,tp,tSub,lSub,rSub,Lcorr,Rcorr,Lpv,Rpv,animal_ID
2,5535877.0,5535988.0,5535932.0,5.0,4.0,4.0,-0.2,0.6,1.0,1.0,3837.0
